In [1]:
from pathlib import Path

import pandas as pd
import pyomo.environ as pyo

In [2]:
menu_file_path = Path("..") / "data" / "menu.csv"

In [3]:
columns = [
    "Category",
    "Item",
    "Calories",
]

upper_bound_constraints = {
    "Total Fat": 78,
    "Saturated Fat": 20,
    # "Trans Fat": ???,
    "Cholesterol": 300,
    "Sodium": 2_300,
    "Sugars": 50,
}

lower_bound_constraints = {
    "Carbohydrates": 275,
    "Dietary Fiber": 28,
    "Protein": 50,
}

all_cols = (columns
            + list(upper_bound_constraints.keys())
            + list(lower_bound_constraints.keys()))

In [4]:
menu = pd.read_csv(
    menu_file_path,
    usecols=all_cols,
    index_col="Item"
)

In [5]:
# Filter out rows with no calories
# This can't be the case as all nutrients contain some caloric value
menu = menu[menu["Calories"] > 1e-6]

In [6]:
menu.head()

,Category,Calories,Total Fat,Saturated Fat,Cholesterol,Sodium,Carbohydrates,Dietary Fiber,Sugars,Protein
Item,,,,,,,,,,
Egg McMuffin,Breakfast,300,13.0,5.0,260,750,31,4,3,17
Egg White Delight,Breakfast,250,8.0,3.0,25,770,30,4,3,18
Sausage McMuffin,Breakfast,370,23.0,8.0,45,780,29,4,2,14
Sausage McMuffin with Egg,Breakfast,450,28.0,10.0,285,860,30,4,2,21
Sausage McMuffin with Egg Whites,Breakfast,400,23.0,8.0,50,880,30,4,2,21


In [7]:
m = pyo.ConcreteModel()
m.x = pyo.Var([x for x in menu.index], domain=pyo.NonNegativeIntegers)

In [8]:
m.obj_func = pyo.Objective(expr=pyo.sum_product(menu["Calories"],  m.x))

In [9]:
def prettify_constraint_name(constraint_name: str) -> str:
    return constraint_name.replace(" ", "_").lower()


In [10]:
for name, constraint_value in upper_bound_constraints.items():
    constraint_name = f"constraint_{prettify_constraint_name(name)}"
    expr = pyo.sum_product(menu[name], m.x) <= constraint_value
    constraint = pyo.Constraint(expr=expr)
    setattr(m, constraint_name, constraint)

In [11]:
for name, constraint_value in lower_bound_constraints.items():
    constraint_name = prettify_constraint_name(name)
    expr = pyo.sum_product(menu[name], m.x) >= constraint_value
    constraint = pyo.Constraint(expr=expr)
    setattr(m, constraint_name, constraint)

In [12]:
solver = pyo.SolverFactory("appsi_highs")

In [13]:
_ = solver.solve(m)

In [14]:
def get_basic_variables(model: pyo.ConcreteModel) -> dict[str, float]:
    basic_variables = {}
    for key in model.x.keys():
        if abs(model.x[key].value) > 1e-6:
            basic_variables[key] = model.x[key].value
    return basic_variables

In [15]:
get_basic_variables(m)

{'Egg White Delight': 1.0,
 'Hotcakes': 1.0,
 'Large French Fries': 1.0,
 'Kids French Fries': 3.0,
 'Side Salad': 16.0,
 'Iced Coffee with Sugar Free French Vanilla Syrup (Small)': 1.0}

In [16]:
m.constraint_less_than_5_per_item = pyo.ConstraintList()
for index in menu.index:
    m.constraint_less_than_5_per_item.add(m.x[index] <= 4)

In [17]:
_ = solver.solve(m)

In [18]:
get_basic_variables(m)

{'Egg White Delight': 1.0,
 'Hotcakes': 1.0,
 'Fruit & Maple Oatmeal without Brown Sugar': 1.0,
 'Premium Southwest Salad (without Chicken)': 1.0,
 'Medium French Fries': 1.999999999997543,
 'Kids French Fries': 0.9999999999998779,
 'Side Salad': 3.9999999999998086}

In [19]:
def constrain_category_by_calories(model: pyo.ConcreteModel, df: pd.DataFrame, min_calories: int, category: str) -> None:
    constraint_name = f"constraint_min_cals_{category}"
    category_df = df[df["Category"] == category]
    expr = sum(category_df.loc[i, "Calories"] * model.x[i] for i in category_df.index) >= min_calories
    setattr(model, constraint_name, expr)

In [20]:
constrain_category_by_calories(m, menu, 250, "Breakfast")
constrain_category_by_calories(m, menu, 500, "Beef & Pork")
constrain_category_by_calories(m, menu, 500, "Chicken & Fish")

In [21]:
_ = solver.solve(m)

In [22]:
get_basic_variables(m)

{'Egg White Delight': 1.0,
 'Hotcakes': 1.0,
 'Fruit & Maple Oatmeal without Brown Sugar': 1.0,
 'Premium Southwest Salad (without Chicken)': 1.0,
 'Medium French Fries': 2.0000000000000346,
 'Kids French Fries': 0.9999999999998614,
 'Side Salad': 4.0}

1 Var Declarations
    x : Size=244, Index={Egg McMuffin, Egg White Delight, Sausage McMuffin, Sausage McMuffin with Egg, Sausage McMuffin with Egg Whites, Steak & Egg McMuffin, Bacon, Egg & Cheese Biscuit (Regular Biscuit), Bacon, Egg & Cheese Biscuit (Large Biscuit), Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit), Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit), Sausage Biscuit (Regular Biscuit), Sausage Biscuit (Large Biscuit), Sausage Biscuit with Egg (Regular Biscuit), Sausage Biscuit with Egg (Large Biscuit), Sausage Biscuit with Egg Whites (Regular Biscuit), Sausage Biscuit with Egg Whites (Large Biscuit), Southern Style Chicken Biscuit (Regular Biscuit), Southern Style Chicken Biscuit (Large Biscuit), Steak & Egg Biscuit (Regular Biscuit), Bacon, Egg & Cheese McGriddles, Bacon, Egg & Cheese McGriddles with Egg Whites, Sausage McGriddles, Sausage, Egg & Cheese McGriddles, Sausage, Egg & Cheese McGriddles with Egg Whites, Bacon, Egg & Cheese Bagel, Baco